# Sports News Objectivity Data

Multivariate, integer attributes, classification problem. 1000 instances, 59 attributes, no missing data. We are trying to label the text objective or subjective.

1000 sports articles were labeled using Amazon Mechanical Turk as objective or subjective. The raw texts, extracted features, and the URLs from which the articles were retrieved are provided.

In [4]:
library(glmnet)
library(tidyverse)
library(caret)
library(pROC)
library(randomForest)
library(gbm)
library(rpart)
require(rpart.plot)
library(e1071)
library(ranger)

In [3]:
sports <- read.table("C:/Users/n__gu/Desktop/sports.txt", header=TRUE)
dim(sports)
str(sports)

[1] 1000   61

'data.frame':	1000 obs. of  61 variables:
 $ TextID           : Factor w/ 1000 levels "Text0001","Text0002",..: 1 2 3 4 5 6 7 8 9 11 ...
 $ totalWordsCount  : int  109 309 149 305 491 314 314 462 808 860 ...
 $ semanticobjscore : int  0 21 6 18 23 14 10 19 40 44 ...
 $ semanticsubjscore: int  1 4 1 5 8 1 6 6 11 22 ...
 $ CC               : int  7 1 8 7 33 17 1 5 49 13 ...
 $ CD               : int  9 19 14 26 47 17 37 47 71 111 ...
 $ DT               : int  0 1 0 0 0 0 1 0 1 0 ...
 $ EX               : int  5 4 5 10 12 0 10 0 6 4 ...
 $ FW               : int  8 35 15 37 61 36 33 40 94 101 ...
 $ INs              : int  6 23 11 21 36 15 20 39 44 56 ...
 $ JJ               : int  0 0 0 1 0 2 0 0 2 3 ...
 $ JJR              : int  0 0 0 1 1 1 0 1 1 2 ...
 $ JJS              : int  0 0 0 0 0 0 0 0 0 0 ...
 $ LS               : int  0 2 0 0 2 0 0 9 0 2 ...
 $ MD               : int  29 93 47 83 142 88 112 101 162 196 ...
 $ NN               : int  0 1 1 2 1 1 3 1 7 2 ...
 $ NNP           

In [5]:
sports <- subset(sports, select=-c(TextID))


In [6]:
dim(sports)

[1] 1000   60

In [7]:
set.seed(101) # Set Seed so that same sample can be reproduced in future also
# Now Selecting 75% of data as sample from total 'n' rows of the data  
sample <- sample.int(n = nrow(sports), size = floor(.75*nrow(sports)), replace = F)
train <- sports[sample, ]
test  <- sports[-sample, ]

## Lasso Regression

LASSO regression, find best lambda parameter with cross validation.

In [8]:
train <- as.data.frame(train)
x <- as.matrix(train[,1:59])
y <- as.matrix(as.factor(train[,60]))

In [9]:
set.seed(123) 
cv <- cv.glmnet(x, y, alpha = 1,family="binomial")

# Display the best lambda value
cv$lambda.min
model <- glmnet(x, y , alpha = 1, lambda = cv$lambda.min,family="binomial")
# Display regression coefficients
coef(model)


[1] 0.009778432

60 x 1 sparse Matrix of class "dgCMatrix"
                             s0
(Intercept)       -1.3509583314
totalWordsCount    .           
semanticobjscore   .           
semanticsubjscore  0.0029512177
CC                -0.0163373261
CD                 0.0022222168
DT                 0.0727121152
EX                 .           
FW                 .           
INs                .           
JJ                 .           
JJR                .           
JJS                0.4243318899
LS                 0.0059703453
MD                 .           
NN                -0.0105248178
NNP                .           
NNPS               .           
NNS                .           
PDT               -0.0481510974
POS                .           
PRP                .           
PRP.               0.0199072021
RB                 0.1440880966
RBR                .           
RBS                .           
RP                 .           
SYM                .           
TOs                0.218309559

In [11]:
x.train <- as.matrix(train[,1:59])
predictions <- model %>% predict(x.train) %>% as.vector()

table(train[,60])
auc(roc(response = train$Label, predictor = predictions))



 objective subjective 
       476        274 

Setting levels: control = objective, case = subjective
Setting direction: controls < cases


Area under the curve: 0.8972

In [12]:
x.test <- as.matrix(test[,1:59])
predictions <- model %>% predict(x.test) %>% as.vector()

table(test[,60])
auc(roc(response = test$Label, predictor = predictions))


 objective subjective 
       159         91 

Setting levels: control = objective, case = subjective
Setting direction: controls < cases


Area under the curve: 0.901

Worked well, AUC=0.9 for test and 0.89 for train data shows good performance. No issue of overfitting.

## Decision Tree

Best parameters of the minimal number of observations per tree leaf and complexity parameter.

In [13]:
caret.control <- trainControl(method = "repeatedcv",
                              number = 10,
                              repeats = 3)

In [14]:
cpGrid = expand.grid( .cp = seq(0.01,0.5) )


In [15]:
rpart.cv <- train(Label ~ ., 
                  data = train,
                  method = "rpart",
                  trControl = caret.control,
                  tuneGrid = cpGrid)

In [16]:
rpart.cv

CART 

750 samples
 59 predictor
  2 classes: 'objective', 'subjective' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 3 times) 
Summary of sample sizes: 675, 675, 674, 675, 675, 674, ... 
Resampling results:

  Accuracy   Kappa    
  0.7866758  0.5321262

Tuning parameter 'cp' was held constant at a value of 0.01

In [17]:
tree2 <- rpart(Label~., data=train,control = rpart.control(minbucket =2, cp= 0.02) , method = 'class')
predict2 <-predict(tree2, newdata=test, type = 'class')
confusionMatrix(test$Label,predict2)

Confusion Matrix and Statistics

            Reference
Prediction   objective subjective
  objective        133         26
  subjective        19         72
                                          
               Accuracy : 0.82            
                 95% CI : (0.7667, 0.8656)
    No Information Rate : 0.608           
    P-Value [Acc > NIR] : 3.735e-13       
                                          
                  Kappa : 0.6175          
                                          
 Mcnemar's Test P-Value : 0.3711          
                                          
            Sensitivity : 0.8750          
            Specificity : 0.7347          
         Pos Pred Value : 0.8365          
         Neg Pred Value : 0.7912          
             Prevalence : 0.6080          
         Detection Rate : 0.5320          
   Detection Prevalence : 0.6360          
      Balanced Accuracy : 0.8048          
                                          
       'Positive' Class : 

In [45]:
tree2 <- rpart(Label~., data=train,control = rpart.control(minbucket =5, cp= 0.02) , method = 'class')
predict2 <-predict(tree2, newdata=test, type = 'class')
confusionMatrix(test$Label,predict2)

Confusion Matrix and Statistics

            Reference
Prediction   objective subjective
  objective        133         26
  subjective        19         72
                                          
               Accuracy : 0.82            
                 95% CI : (0.7667, 0.8656)
    No Information Rate : 0.608           
    P-Value [Acc > NIR] : 3.735e-13       
                                          
                  Kappa : 0.6175          
                                          
 Mcnemar's Test P-Value : 0.3711          
                                          
            Sensitivity : 0.8750          
            Specificity : 0.7347          
         Pos Pred Value : 0.8365          
         Neg Pred Value : 0.7912          
             Prevalence : 0.6080          
         Detection Rate : 0.5320          
   Detection Prevalence : 0.6360          
      Balanced Accuracy : 0.8048          
                                          
       'Positive' Class : 

In [26]:
tree2 <- rpart(Label~., data=train,control = rpart.control(minbucket =3, cp= 0.015) , method = 'class')
predict2 <-predict(tree2, newdata=test, type = 'class')
confusionMatrix(test$Label,predict2)

Confusion Matrix and Statistics

            Reference
Prediction   objective subjective
  objective        133         26
  subjective        19         72
                                          
               Accuracy : 0.82            
                 95% CI : (0.7667, 0.8656)
    No Information Rate : 0.608           
    P-Value [Acc > NIR] : 3.735e-13       
                                          
                  Kappa : 0.6175          
                                          
 Mcnemar's Test P-Value : 0.3711          
                                          
            Sensitivity : 0.8750          
            Specificity : 0.7347          
         Pos Pred Value : 0.8365          
         Neg Pred Value : 0.7912          
             Prevalence : 0.6080          
         Detection Rate : 0.5320          
   Detection Prevalence : 0.6360          
      Balanced Accuracy : 0.8048          
                                          
       'Positive' Class : 

In [25]:
tree2 <- rpart(Label~., data=train,control = rpart.control(minbucket =3, cp= 0.02) , method = 'class')
predict2 <-predict(tree2, newdata=test, type = 'class')
confusionMatrix(test$Label,predict2)

Confusion Matrix and Statistics

            Reference
Prediction   objective subjective
  objective        133         26
  subjective        19         72
                                          
               Accuracy : 0.82            
                 95% CI : (0.7667, 0.8656)
    No Information Rate : 0.608           
    P-Value [Acc > NIR] : 3.735e-13       
                                          
                  Kappa : 0.6175          
                                          
 Mcnemar's Test P-Value : 0.3711          
                                          
            Sensitivity : 0.8750          
            Specificity : 0.7347          
         Pos Pred Value : 0.8365          
         Neg Pred Value : 0.7912          
             Prevalence : 0.6080          
         Detection Rate : 0.5320          
   Detection Prevalence : 0.6360          
      Balanced Accuracy : 0.8048          
                                          
       'Positive' Class : 

In [18]:
predict2 <-predict(tree2, newdata=train, type = 'class')
confusionMatrix(train$Label,predict2)

Confusion Matrix and Statistics

            Reference
Prediction   objective subjective
  objective        416         60
  subjective        64        210
                                          
               Accuracy : 0.8347          
                 95% CI : (0.8061, 0.8606)
    No Information Rate : 0.64            
    P-Value [Acc > NIR] : <2e-16          
                                          
                  Kappa : 0.6424          
                                          
 Mcnemar's Test P-Value : 0.7876          
                                          
            Sensitivity : 0.8667          
            Specificity : 0.7778          
         Pos Pred Value : 0.8739          
         Neg Pred Value : 0.7664          
             Prevalence : 0.6400          
         Detection Rate : 0.5547          
   Detection Prevalence : 0.6347          
      Balanced Accuracy : 0.8222          
                                          
       'Positive' Class : 

In [27]:
set.seed(123)

errs <- rep(NA, length(folds))

for (i in 1:10) {
 tmp.model <- rpart(Label~. , train,control = rpart.control(minbucket =3, cp= 0.02), method = "class")
 tmp.predict <- predict(tmp.model, newdata = test, type = "class")
 conf.mat <- table(test$Label, tmp.predict)
 errs[i] <- 1-sum(diag(conf.mat))/sum(conf.mat)
}
print(sprintf("average error using k-fold cross-validation: %.3f percent", 100*mean(errs)))

[1] "average error using k-fold cross-validation: 18.000 percent"


In [26]:
set.seed(123)

errs <- rep(NA, length(folds))

for (i in 1:10) {
 tmp.model <- rpart(Label~. , train,control = rpart.control(minbucket =3, cp= 0.02), method = "class")
 tmp.predict <- predict(tmp.model, newdata = train, type = "class")
 conf.mat <- table(train$Label, tmp.predict)
 errs[i] <- 1-sum(diag(conf.mat))/sum(conf.mat)
}
print(sprintf("average error using k-fold cross-validation: %.3f percent", 100*mean(errs)))

[1] "average error using k-fold cross-validation: 16.533 percent"


After trying diffferent set of parameters, the best parameters are found to be minbucket = 3, cp = 0.02. Accuracy is 0.82, somewhat good result.

Average error of k-fold cross valiation is 18% for test data, and 16.5% for train data. Accuracy for train data is 0.83. We do not observe under or overfitting.


## Random Forest

Best parameter for the minimal number of observations per tree leaf with cross validation. Min node size is set to 5 and ntree to 500.

In [19]:
folds <- createMultiFolds(train$Label, k = 10, times = 3)
control <- trainControl(method = "cv", number = 10, verboseIter = TRUE,
                        classProbs = TRUE, savePredictions = TRUE, index = folds, allowParallel = TRUE)
tune_grid <- expand.grid(mtry = c(2,5,7,10), splitrule = "gini", min.node.size = 5)
rf <- train(Label~., train, method = "ranger", tuneGrid = tune_grid, trControl = control)

+ Fold01.Rep1: mtry= 2, splitrule=gini, min.node.size=5 
- Fold01.Rep1: mtry= 2, splitrule=gini, min.node.size=5 
+ Fold01.Rep1: mtry= 5, splitrule=gini, min.node.size=5 
- Fold01.Rep1: mtry= 5, splitrule=gini, min.node.size=5 
+ Fold01.Rep1: mtry= 7, splitrule=gini, min.node.size=5 
- Fold01.Rep1: mtry= 7, splitrule=gini, min.node.size=5 
+ Fold01.Rep1: mtry=10, splitrule=gini, min.node.size=5 
- Fold01.Rep1: mtry=10, splitrule=gini, min.node.size=5 
+ Fold02.Rep1: mtry= 2, splitrule=gini, min.node.size=5 
- Fold02.Rep1: mtry= 2, splitrule=gini, min.node.size=5 
+ Fold02.Rep1: mtry= 5, splitrule=gini, min.node.size=5 
- Fold02.Rep1: mtry= 5, splitrule=gini, min.node.size=5 
+ Fold02.Rep1: mtry= 7, splitrule=gini, min.node.size=5 
- Fold02.Rep1: mtry= 7, splitrule=gini, min.node.size=5 
+ Fold02.Rep1: mtry=10, splitrule=gini, min.node.size=5 
- Fold02.Rep1: mtry=10, splitrule=gini, min.node.size=5 
+ Fold03.Rep1: mtry= 2, splitrule=gini, min.node.size=5 
- Fold03.Rep1: mtry= 2, splitru

In [20]:
predictrf <-predict(rf, newdata=test)

In [21]:
confusionMatrix(test$Label,predictrf)

Confusion Matrix and Statistics

            Reference
Prediction   objective subjective
  objective        140         19
  subjective        23         68
                                          
               Accuracy : 0.832           
                 95% CI : (0.7798, 0.8762)
    No Information Rate : 0.652           
    P-Value [Acc > NIR] : 1.952e-10       
                                          
                  Kappa : 0.6337          
                                          
 Mcnemar's Test P-Value : 0.6434          
                                          
            Sensitivity : 0.8589          
            Specificity : 0.7816          
         Pos Pred Value : 0.8805          
         Neg Pred Value : 0.7473          
             Prevalence : 0.6520          
         Detection Rate : 0.5600          
   Detection Prevalence : 0.6360          
      Balanced Accuracy : 0.8203          
                                          
       'Positive' Class : 

In [22]:
predictrf <-predict(rf, newdata=train)
confusionMatrix(train$Label,predictrf)

Confusion Matrix and Statistics

            Reference
Prediction   objective subjective
  objective        476          0
  subjective        12        262
                                          
               Accuracy : 0.984           
                 95% CI : (0.9722, 0.9917)
    No Information Rate : 0.6507          
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.9652          
                                          
 Mcnemar's Test P-Value : 0.001496        
                                          
            Sensitivity : 0.9754          
            Specificity : 1.0000          
         Pos Pred Value : 1.0000          
         Neg Pred Value : 0.9562          
             Prevalence : 0.6507          
         Detection Rate : 0.6347          
   Detection Prevalence : 0.6347          
      Balanced Accuracy : 0.9877          
                                          
       'Positive' Class : 

Accuracy is 0.83 for test data, and 0.98 for train data. We observe overfitting.

## Stochastic Gradient Boosting

Best parameters for depth, learning rate, and number of trees.

In [23]:
folds <- createMultiFolds(train$Label, k = 10, times = 1)
control <- trainControl(method = "cv", number = 10, verboseIter = TRUE, summaryFunction = twoClassSummary,
                        classProbs = TRUE, savePredictions = TRUE, index = folds, allowParallel = TRUE)
tune_grid <- expand.grid(interaction.depth=c(1, 3, 5), n.trees = c(500, 1000), shrinkage=c(0.01, 0.001), n.minobsinnode = 10)
gbm <- train(Label ~., data = train, method = "gbm", metric = "Accuracy", tuneGrid = tune_grid, trControl = control)

Warning message in train.default(x, y, weights = w, ...):
"The metric "Accuracy" was not in the result set. ROC will be used instead."

+ Fold01.Rep1: shrinkage=0.001, interaction.depth=1, n.minobsinnode=10, n.trees=1000 


Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"variable 16: NNP has no variation."Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"variable 39: WRB has no variation."Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"variable 48: ellipsis has no variation."

Iter   TrainDeviance   ValidDeviance   StepSize   Improve
     1        1.3123             nan     0.0010    0.0003
     2        1.3117             nan     0.0010    0.0003
     3        1.3110             nan     0.0010    0.0003
     4        1.3104             nan     0.0010    0.0003
     5        1.3098             nan     0.0010    0.0003
     6        1.3092             nan     0.0010    0.0003
     7        1.3086             nan     0.0010    0.0003
     8        1.3080             nan     0.0010    0.0003
     9        1.3075             nan     0.0010    0.0003
    10        1.3069             nan     0.0010    0.0003
    20        1.3008             nan     0.0010    0.0003
    40        1.2890             nan     0.0010    0.0003
    60        1.2779             nan     0.0010    0.0002
    80        1.2672             nan     0.0010    0.0003
   100        1.2567             nan     0.0010    0.0002
   120        1.2465             nan     0.0010    0.0002
   140        

Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"variable 16: NNP has no variation."Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"variable 39: WRB has no variation."Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"variable 48: ellipsis has no variation."

...

Iter   TrainDeviance   ValidDeviance   StepSize   Improve
     1        1.3051             nan     0.0100    0.0035
     2        1.2979             nan     0.0100    0.0032
     3        1.2915             nan     0.0100    0.0027
     4        1.2838             nan     0.0100    0.0038
     5        1.2768             nan     0.0100    0.0031
     6        1.2695             nan     0.0100    0.0027
     7        1.2632             nan     0.0100    0.0029
     8        1.2563             nan     0.0100    0.0029
     9        1.2495             nan     0.0100    0.0033
    10        1.2428             nan     0.0100    0.0029
    20        1.1819             nan     0.0100    0.0022
    40        1.0875             nan     0.0100    0.0019
    60        1.0158             nan     0.0100    0.0014
    80        0.9592             nan     0.0100    0.0010
   100        0.9152             nan     0.0100    0.0009
   120        0.8801             nan     0.0100    0.0004
   140        

Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"variable 16: NNP has no variation."Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"variable 39: WRB has no variation."Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"variable 48: ellipsis has no variation."

Iter   TrainDeviance   ValidDeviance   StepSize   Improve
     1        1.3039             nan     0.0100    0.0037
     2        1.2960             nan     0.0100    0.0036
     3        1.2876             nan     0.0100    0.0038
     4        1.2789             nan     0.0100    0.0038
     5        1.2712             nan     0.0100    0.0029
     6        1.2631             nan     0.0100    0.0034
     7        1.2554             nan     0.0100    0.0028
     8        1.2474             nan     0.0100    0.0033
     9        1.2399             nan     0.0100    0.0032
    10        1.2323             nan     0.0100    0.0031
    20        1.1686             nan     0.0100    0.0025
    40        1.0628             nan     0.0100    0.0021
    60        0.9812             nan     0.0100    0.0014
    80        0.9187             nan     0.0100    0.0010
   100        0.8686             nan     0.0100    0.0006
   120        0.8277             nan     0.0100    0.0005
   140        

Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"variable 16: NNP has no variation."Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"variable 39: WRB has no variation."Warning message in (function (x, y, offset = NULL, misc = NULL, distribution = "bernoulli", :
"variable 48: ellipsis has no variation."

Iter   TrainDeviance   ValidDeviance   StepSize   Improve
     1        1.3055             nan     0.0100    0.0035
     2        1.2977             nan     0.0100    0.0034
     3        1.2902             nan     0.0100    0.0032
     4        1.2826             nan     0.0100    0.0037
     5        1.2749             nan     0.0100    0.0033
     6        1.2674             nan     0.0100    0.0033
     7        1.2603             nan     0.0100    0.0033
     8        1.2534             nan     0.0100    0.0030
     9        1.2466             nan     0.0100    0.0031
    10        1.2400             nan     0.0100    0.0032
    20        1.1808             nan     0.0100    0.0024
    40        1.0851             nan     0.0100    0.0020
    60        1.0132             nan     0.0100    0.0013
    80        0.9554             nan     0.0100    0.0008
   100        0.9099             nan     0.0100    0.0008
   120        0.8738             nan     0.0100    0.0004
   140        

In [24]:
gbm

Stochastic Gradient Boosting 

750 samples
 59 predictor
  2 classes: 'objective', 'subjective' 

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 676, 675, 675, 675, 676, 674, ... 
Resampling results across tuning parameters:

  shrinkage  interaction.depth  n.trees  ROC        Sens       Spec     
  0.001      1                   500     0.8541571  0.9475177  0.4525132
  0.001      1                  1000     0.8556007  0.9013741  0.6238095
  0.001      3                   500     0.8701393  0.9558954  0.4994709
  0.001      3                  1000     0.8717922  0.9034574  0.6789683
  0.001      5                   500     0.8749237  0.9643174  0.4921958
  0.001      5                  1000     0.8781604  0.9118351  0.6862434
  0.010      1                   500     0.8655422  0.8845301  0.7191799
  0.010      1                  1000     0.8738009  0.8844858  0.7227513
  0.010      3                   500     0.8817070  0.8929965  0.7373016
  0.010  

In [25]:
predictions <- predict(gbm, newdata = test)
confusionMatrix(test$Label,predictions)

Confusion Matrix and Statistics

            Reference
Prediction   objective subjective
  objective        142         17
  subjective        20         71
                                          
               Accuracy : 0.852           
                 95% CI : (0.8018, 0.8936)
    No Information Rate : 0.648           
    P-Value [Acc > NIR] : 4.153e-13       
                                          
                  Kappa : 0.6781          
                                          
 Mcnemar's Test P-Value : 0.7423          
                                          
            Sensitivity : 0.8765          
            Specificity : 0.8068          
         Pos Pred Value : 0.8931          
         Neg Pred Value : 0.7802          
             Prevalence : 0.6480          
         Detection Rate : 0.5680          
   Detection Prevalence : 0.6360          
      Balanced Accuracy : 0.8417          
                                          
       'Positive' Class : 

In [26]:
predictions <- predict(gbm, newdata = train)
confusionMatrix(train$Label,predictions)

Confusion Matrix and Statistics

            Reference
Prediction   objective subjective
  objective        442         34
  subjective        58        216
                                       
               Accuracy : 0.8773       
                 95% CI : (0.8517, 0.9)
    No Information Rate : 0.6667       
    P-Value [Acc > NIR] : < 2e-16      
                                       
                  Kappa : 0.7305       
                                       
 Mcnemar's Test P-Value : 0.01649      
                                       
            Sensitivity : 0.8840       
            Specificity : 0.8640       
         Pos Pred Value : 0.9286       
         Neg Pred Value : 0.7883       
             Prevalence : 0.6667       
         Detection Rate : 0.5893       
   Detection Prevalence : 0.6347       
      Balanced Accuracy : 0.8740       
                                       
       'Positive' Class : objective    
                                       

Accuracy for test data is 0.85 and 0.87 for train data. Worked well and no issue of overfitting.

With 90% LASSO performed the best and SGB is second with 0.85. Random Forest was 0.83 but has overfitting. Decision tree has 0.82. For this dataset, LASSO is the best performer, high accuracy but no overfitting.